# Contrails and contrail cirrus

Basic method follows [Lee et al. (2020)](https://www.sciencedirect.com/science/article/pii/S1352231020305689). We extend beyond the end of the Lee assessment (2018) by using:

- Lee et al. (2020); there is a supplementary Excel dataset that details all of the calculations, including the conversion between distance travelled and contrail radiative forcing (a constant value); the nub is therefore extracting the distance travelled from post-2018 data. We have fule consumption estimates as follows:

- Sausen and Schumann (2000) for the 1940-1970 period.
  
- IEA World Oil Statistics, Edition 2023, for World region and the sum of DAVGAS, DJETGAS and DJETKERO. We use data for 1971-2021. This is paywalled data so we do not include it in the repository. It is from https://doi.org/10.1787/218759da-en.

- IATA (2023), aviation fuel use, table 6 of https://www.iata.org/en/iata-repository/publications/economic-reports/global-outlook-for-air-transport---december-2023---report/. This data is available to 2023 as an estimate and 2024 as a projection. It could fully replace the IEA data, but we continue to use IEA to follow Lee.

In Lee et al. (2020), contrail ERF is calculated as:

distance (km) * scale factor (1.17) * contrail forcing per km (see below)

The scale factor converts horizontal distance travelled to 3D distance.

Lee et al assume a constant fuel consumption (CO2 emission per fuel burn) of 3.16 kgCO2 per kg fuel

From Wiki (https://en.wikipedia.org/wiki/Aviation_fuel) I get 0.81 kg/L (accessed 2024-02-27)

RPKs, ASKs and distances from https://www.airlines.org/dataset/world-airlines-traffic-and-capacity/ for 1950-2021

## recipe
1. read in Lee spreadsheet
2. derive contrails per kilogram fuel burn
3. calculate IEA factor offline and apply - read in this time series
4. apply factor to IATA data
5. check overlaps and construct time series

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

In [ ]:
kg_co2_per_kg_fuel = 3.16

In [ ]:
iata = pd.DataFrame(np.array([359, 196, 236, 292, 357, 377])*1e9, index=pd.Index(np.arange(2019, 2025), name='year'), columns=['Aviation fuel, litres'])

In [ ]:
iata

In [ ]:
# according to Wiki, aviation fuel has a density of around 0.81 kg/L, so convert to kilograms
kg_per_litre = 0.81
iata['Aviation fuel, kg'] = iata['Aviation fuel, litres'] * kg_per_litre
iata['Aviation emissions, MtCO2/yr'] = iata['Aviation fuel, kg'] * kg_co2_per_kg_fuel / 1e9
iata

In [ ]:
iea_fuel_raw = pd.read_csv('../data/contrails/OIWORLD-2023-1-EN-20231014T121149.csv')

In [ ]:
iea = np.zeros(2022-1971)
for iyear, year in enumerate(range(1971, 2022)):
    iea[iyear] = iea_fuel_raw[iea_fuel_raw["TIME"]==year].Value.sum()

In [ ]:
iea = pd.DataFrame(iea, index=pd.Index(np.arange(1971, 2022), name='year'), columns=['Aviation fuel, kilotons'])
iea['Aviation fuel, kg'] = iea['Aviation fuel, kilotons'] * 1e6
iea['Aviation emissions, MtCO2/yr'] = iea['Aviation fuel, kg'] * kg_co2_per_kg_fuel / 1e9
iea

In [ ]:
sausen_schumann = pd.read_csv('../data/contrails/sausen_schumann_co2.csv', index_col=0)

In [ ]:
sausen_schumann

In [ ]:
# make a column in CO2 units, which is comparible directly to Lee et al.
sausen_schumann['Aviation emissions, MtCO2/yr'] = sausen_schumann['aviation_emissions_teragrams_carbon'] * 3.664
sausen_schumann

In [ ]:
lee2020 = pd.read_csv('../data/contrails/lee2020_clean_data.csv', index_col=0)
lee2020.rename(columns={'CO2 Tg/yr': 'Aviation emissions, MtCO2/yr'}, inplace=True)

In [ ]:
pl.plot(sausen_schumann['Aviation emissions, MtCO2/yr'])
pl.plot(iea['Aviation emissions, MtCO2/yr'])
pl.plot(lee2020['Aviation emissions, MtCO2/yr'])
pl.plot(iata['Aviation emissions, MtCO2/yr'])

In [ ]:
constructed_emissions = pd.DataFrame(np.ones(94)*np.nan, index=pd.Index(np.arange(1930, 2024), name='year'), columns=['MtCO2/yr'])

In [ ]:
constructed_emissions.loc[1940:1970, 'MtCO2/yr'] = sausen_schumann.loc[1940:1970, 'Aviation emissions, MtCO2/yr']
constructed_emissions.loc[1971:2021, 'MtCO2/yr'] = iea.loc[1971:2021, 'Aviation emissions, MtCO2/yr']
constructed_emissions.loc[2022:2023, 'MtCO2/yr'] = iata.loc[2022:2023, 'Aviation emissions, MtCO2/yr']
constructed_emissions.loc[1930, 'MtCO2/yr'] = 0
constructed_emissions.interpolate(inplace=True)

In [ ]:
pl.plot(constructed_emissions)

In [ ]:
pl.plot(lee2020['Scaled distance million km']/lee2020['Aviation emissions, MtCO2/yr']/ 1000)
pl.title('Fuel efficiency')
pl.ylabel('km / kgCO2')

In [ ]:
constructed_efficiency = pd.DataFrame(np.ones(94) * np.nan, index=pd.Index(np.arange(1930, 2024), name='year'), columns=['km / kgCO2'])
constructed_efficiency.loc[1990:2018, 'km / kgCO2'] = lee2020['Scaled distance million km']/lee2020['Aviation emissions, MtCO2/yr']/ 1000
#constructed_efficiency[2019] = 
# For 2019 to 2021, take estimates of distance from https://www.airlines.org/dataset/world-airlines-traffic-and-capacity/
# scale up by 1.17 as in Lee
constructed_efficiency.loc[2019, 'km / kgCO2'] = 1.17 * 56199 / iea.loc[2019, 'Aviation emissions, MtCO2/yr'] / 1000
constructed_efficiency.loc[2020, 'km / kgCO2'] = 1.17 * 28013 / iea.loc[2020, 'Aviation emissions, MtCO2/yr'] / 1000
constructed_efficiency.loc[2021, 'km / kgCO2'] = 1.17 * 33705 / iea.loc[2021, 'Aviation emissions, MtCO2/yr'] / 1000

# for 2022 and 2023, we have no better means than using 2021 data. But really, we'd expect efficiency to improve, since there were probably
# fewer half-empty planes after COVID.
constructed_efficiency.loc[2022:2023, 'km / kgCO2'] = constructed_efficiency.loc[2021, 'km / kgCO2']

Now try to port this backwards

Lee et al: "Aviation transport efficiency has improved by approximately eightfold since 1960". I know this is based on RPKs rather than actual kilometers.

According to https://www.airlines.org/dataset/world-airlines-traffic-and-capacity/, capacity increased from about 60% in 1960 to about 80% in 2018/2019.

So 1/3 of the improvement is due to increasing capacity, and 2/3 * 8 is due to aircraft efficiency.

In [ ]:
constructed_efficiency.loc[1990, 'km / kgCO2'] / constructed_efficiency.loc[2018, 'km / kgCO2']

In [ ]:
constructed_efficiency.loc[1960, 'km / kgCO2'] = constructed_efficiency.loc[2018, 'km / kgCO2'] / (8 * 2/3)
constructed_efficiency.loc[1930, 'km / kgCO2'] = constructed_efficiency.loc[1960, 'km / kgCO2']
#constructed_efficiency.loc[1989, 'km / kgCO2'] = constructed_efficiency.loc[2018, 'km / kgCO2'] * 9/16
constructed_efficiency.interpolate(inplace=True)

In [ ]:
pl.plot(constructed_efficiency)

In [ ]:
constructed_efficiency

In [ ]:
constructed_scaled_distance = pd.DataFrame(np.ones(94) * np.nan, index=pd.Index(np.arange(1930, 2024), name='year'), columns=['Scaled distance, km'])
constructed_scaled_distance.loc[1930:2023, 'Scaled distance, km'] = (constructed_emissions.values * constructed_efficiency.values).squeeze() * 1e9
pl.plot(constructed_scaled_distance)

In [ ]:
lee_conversion_km_to_contrail_erf = 9.3595037832885E-13  # cell J73 of supplementary Excel, divide by 1000 to convert milliwatts to watts

In [ ]:
# Do we keep the 2018 value in Lee (0.0574) by scaling this, or use the slightly newer data from IEA? TBF it is the same to 3 d.p.
constructed_scaled_distance.loc[2018, 'Scaled distance, km'] * lee_conversion_km_to_contrail_erf

In [ ]:
# contrail forcing ERF
contrails_erf = pd.DataFrame(np.ones(94) * np.nan, index=pd.Index(np.arange(1930, 2024), name='year'), columns=['Contrails ERF, W/m2'])
contrails_erf.loc[1930:2023, 'Contrails ERF, W/m2'] = (constructed_scaled_distance * lee_conversion_km_to_contrail_erf).values.squeeze()

pl.plot(contrails_erf)

In [ ]:
contrails_erf.to_csv('../output/contrails_ERF_1930-2023.csv')